In [ ]:
import os

import matplotlib
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import ticker
from scipy.stats import circmean, circstd

from fit_diagnostics import abs_residuals
from plot_utils import antpos_map, df_heatmap, plot_res, plot_res_grouped, plot_res_heatmap
from red_likelihood import flt_ant_pos, makeCArray, split_rel_results
from red_utils import find_rel_df, flt_ant_coords

In [ ]:
%matplotlib inline

In [ ]:
jd_time = 2458098.43869
pol = 'ee'
dist = 'gaussian'
dir_path = 'rel_dfs'
coords = 'cartesian'

In [ ]:
plot_figs = False
if plot_figs:
    import matplotlib as mpl
    mpl.rcParams['figure.dpi'] = 300

In [ ]:
rel_df = find_rel_df(jd_time, pol, dist, dir_path)

In [ ]:
with open(os.path.join(dir_path, 'rel_df.{}.{}.md.pkl'.format(jd_time, pol)), 'rb') as f:
    md = pickle.load(f)
    
df = pd.read_pickle(rel_df)
df.sample(5).sort_index()

In [ ]:
# Check results for a given frequency & time integration
test_freq = 612
test_tint = 0

resx = df.loc[(test_freq, test_tint)][5:-2].values.astype(float)
test_vis, test_gains = split_rel_results(resx, md['no_unq_bls'], coords=coords)
test_gamps = np.abs(test_gains)
test_gphases = np.angle(test_gains)
mean_test_gamps = np.mean(test_gamps)
mean_test_gphases = np.mean(test_gphases)

print('Gain amplitude mean is: {}'.format(mean_test_gamps))
print('Gain phase mean is: {}'.format(mean_test_gphases))

In [ ]:
if 'ex_ant' in dir_path:
    # if any additional bad antennas to flag
    add_bad_ants = dir_path.split('ex_ant', 1)[1].split('_')
    add_bad_ants = np.asarray(add_bad_ants, dtype=int)
else:
    add_bad_ants = None

ant_pos = flt_ant_coords(jd_time, md['antpos'], add_bad_ants=add_bad_ants)

In [ ]:
antpos_map(test_gamps, ant_pos, 'Gain amplitudes', center=1)
print('Gain amplitude solutions for frequency channel {} and time integration {} are: '\
      '\n{}'.format(test_freq, test_tint, test_gamps))

In [ ]:
antpos_map(test_gphases, ant_pos, 'Gain phases')
print('Gain phase solutions are:\n{}'.format(test_gphases))

In [ ]:
# Success rate percentage
success_pct = df['success'].sum() / df['success'].size * 100
print('{}% of iterations (freq/tint slice) were succesful'.format(round(success_pct, 2)))

### Number of iterations required

In [ ]:
plot_res(df, 'nit', clip=True)

In [ ]:
plot_res_grouped(df, 'nit', logy=True)

In [ ]:
plot_res_heatmap(df, 'nit', clip=True)

### Log-likelihood

In [ ]:
plot_res(df, 'fun', clip=True, clip_pctile=98)

In [ ]:
plot_res_grouped(df, 'fun', logy=True)

In [ ]:
plot_res_heatmap(df, 'fun', clip=True, clip_pctile=98)

### Residuals

Look at both the real and imaginary of the the median absolute normalized residual

In [ ]:
df[['med_abs_norm_res_Re', 'med_abs_norm_res_Im']] = df.apply(lambda row: \
pd.Series(abs_residuals(row['norm_residual'])), axis=1)

In [ ]:
plot_res_heatmap(df, 'med_abs_norm_res_Re', clip=True, clip_pctile=90)

In [ ]:
plot_res_heatmap(df, 'med_abs_norm_res_Im', clip=True, clip_pctile=90)

Combined residual by adding Re and Im components in quadrature

In [ ]:
df['med_abs_norm_res_comb'] = np.sqrt(df['med_abs_norm_res_Re']**2 + \
                                      df['med_abs_norm_res_Im']**2)

In [ ]:
plot_res_heatmap(df, 'med_abs_norm_res_comb', clip=True, clip_pctile=90)

### Stability of gains, visibility solutions and degenerate parameters

#### Gains

In [ ]:
no_ants = md['no_ants']
no_resid_p = 5 # number of residual parameters in df dataframe
gains_df = df.iloc[:, -no_ants*2-no_resid_p:-no_resid_p]

if coords == 'cartesian':
    gains_df['gamp_mean'] = gains_df.apply(lambda row: \
                            np.mean(np.abs(makeCArray(row[:no_ants*2].values))), \
                            axis=1)
    gains_df['gphase_mean'] = gains_df.apply(lambda row: \
                              np.mean(np.angle(makeCArray(row[:no_ants*2].values))), \
                              axis=1)
if coords == 'polar':
    gains_df['gamp_mean'] = gains_df.apply(lambda row: \
                            np.mean(row[:no_ants*2:2].values), axis=1)
    gains_df['gphase_mean'] = gains_df.apply(lambda row: \
                              np.mean(row[1:no_ants*2:2].values), axis=1)    

In [ ]:
# Expect all of these to be 1, since the gain amplitudes are either constrained or 
# normalized
plot_res(gains_df, 'gamp_mean')

In [ ]:
plot_res(gains_df, 'gphase_mean', ylim=(-np.pi, np.pi))

In [ ]:
plot_res_heatmap(gains_df, 'gphase_mean', vmin=-np.pi, vmax=np.pi)

##### Gain stability at test_freq {{test_freq}}

In [ ]:
if coords == 'cartesian':
    gainsC_df = gains_df.iloc[:, :-2].apply(lambda row: makeCArray(row.values), axis=1)
    gainsC_df = pd.DataFrame(gainsC_df.values.tolist(), index=gainsC_df.index)

    gamps_df = gainsC_df.abs()
    gphases_df = gainsC_df.apply(np.angle)
if coords == 'polar':
    gamps_df = gains_df.iloc[:, :-2:2]
    gphases_df = gains_df.iloc[:, 1:-2:2]
    
gamps_df.columns = np.arange(no_ants) + 1
gphases_df.columns = gamps_df.columns

In [ ]:
df_heatmap(gamps_df.loc[test_freq, :], ybase=5, center=1, cmap='bwr', \
           title='Gain amplitudes for frequency channel {}'.format(test_freq),
           xlabel='Antenna',
           ylabel='Time integration')

In [ ]:
df_heatmap(gphases_df.loc[test_freq, :], ybase=5, center=0, cmap='bwr', vmin=-np.pi, vmax=np.pi,\
           title='Gain phases for frequency channel {}'.format(test_freq),
           xlabel='Antenna',
           ylabel='Time integration')

##### Gain stability across average over frequency channels 600-700

In [ ]:
std_rng = 3
chan_start = 600
chan_end = 700

# Mean over the time integrations for each frequency
avg_gamps_tint = gamps_df.loc[pd.IndexSlice[chan_start:chan_end, :], :].groupby(level=0).mean()
piv = pd.pivot_table(avg_gamps_tint, columns='freq')
vrng = np.ceil(np.nanstd(piv.values)*std_rng*10)/10

df_heatmap(piv, xbase=10, ybase=5, cmap='bwr', center=1, vmin=1-vrng, vmax=1+vrng, \
           title='Mean (over time integrations) gain amplitudes', \
           xlabel='Frequency', \
           ylabel='Time integration')

In [ ]:
avg_gphases_tint = gphases_df.loc[pd.IndexSlice[chan_start:chan_end, :], :].\
                       groupby(level=0).mean()
piv = pd.pivot_table(avg_gphases_tint, columns='freq')

df_heatmap(piv, xbase=10, ybase=5, cmap='bwr', center=0, vmin=-np.pi, vmax=np.pi, \
           title='Mean (over time integrations) gain phases', \
           xlabel='Frequency', \
           ylabel='Time integration')

In [ ]:
gphases_tint = gphases_df.loc[pd.IndexSlice[chan_start:chan_end, :], :].\
               groupby(level=0)
circ_res = gphases_tint.apply(lambda x: circmean(x, low=-np.pi, high=np.pi, \
                                                 axis=0))
cavg_gphases_tint = pd.DataFrame(circ_res.values.tolist(), index=circ_res.index)
piv = pd.pivot_table(cavg_gphases_tint, columns='freq')

df_heatmap(piv, xbase=10, ybase=5, cmap='bwr', center=0, vmin=-np.pi, vmax=np.pi, \
           title='Circular mean (over time integrations) of gain phases', \
           xlabel='Frequency', \
           ylabel='Time integration')

##### Average gains across frequency channels 600-700 and all time integrations

In [ ]:
avg_gamps = gamps_df.loc[pd.IndexSlice[chan_start:chan_end, :], :].mean(axis=0).values
avg_gphases = gphases_df.loc[pd.IndexSlice[chan_start:chan_end, :], :].mean(axis=0).values
circ_mean_gphases = gphases_df.loc[pd.IndexSlice[chan_start:chan_end, :], :].\
    apply(lambda col: circmean(col, low=-np.pi, high=np.pi), axis=0).values

In [ ]:
antpos_map(avg_gamps, ant_pos, 'Mean gain amplitudes', center=1)

In [ ]:
antpos_map(avg_gphases, ant_pos, 'Mean gain phases', center=0)

In [ ]:
antpos_map(circ_mean_gphases, ant_pos, 'Circular mean of gain phases', center=0)

##### Standard deviation of gains across frequency channels 600-700 and all time integrations

In [ ]:
std_gamps = gamps_df.loc[pd.IndexSlice[chan_start:chan_end, :], :].std(axis=0)
std_gphases = gphases_df.loc[pd.IndexSlice[chan_start:chan_end, :], :].std(axis=0)
circ_std_gphases = gphases_df.loc[pd.IndexSlice[chan_start:chan_end, :], :].apply(lambda col: circstd(col), \
                                                                                  axis=0).values

In [ ]:
antpos_map(std_gamps, ant_pos, 'Standard deviation of gain amplitudes', cmap='Oranges')

In [ ]:
antpos_map(std_gphases, ant_pos, 'Standard deviation of gain phases', cmap='Oranges')

In [ ]:
antpos_map(circ_std_gphases, ant_pos, 'Circular standard deviation of gain phases', \
           cmap='Oranges')

#### Visibility solutions

In [ ]:
no_unq_bls = md['no_unq_bls']
no_min_p = 5 # number of columns in df that are attributes of the SciPy OptimizeResult 
vis_df = df.iloc[:, no_min_p:no_unq_bls*2+no_min_p]

if coords == 'cartesian':
    vis_df['vamp_mean'] = vis_df.apply(lambda row: \
                          np.mean(np.abs(makeCArray(row[:no_unq_bls*2].values))), \
                          axis=1)
    vis_df['vphase_mean'] = vis_df.apply(lambda row: \
                            np.mean(np.angle(makeCArray(row[:no_unq_bls*2].values))), \
                            axis=1)
if coords == 'polar':
    vis_df['vamp_mean'] = vis_df.apply(lambda row: \
                              np.mean(np.absolute(row[:no_unq_bls*2:2].values)), axis=1)
    vis_df['vphase_mean'] = vis_df.apply(lambda row: \
                                np.mean(row[1:no_unq_bls*2:2].values), axis=1)

In [ ]:
plot_res(vis_df, 'vamp_mean', clip=True, clip_pctile=98)

In [ ]:
plot_res_heatmap(vis_df, 'vamp_mean', clip=True, clip_pctile=98)

In [ ]:
plot_res(vis_df, 'vphase_mean', clip=False, ylim=(-np.pi, np.pi))

In [ ]:
plot_res_heatmap(vis_df, 'vphase_mean', vmin=-np.pi, vmax=np.pi)

##### Visibilities across baselines at test_integration {{test_tint}}

In [ ]:
if coords == 'cartesian':
    visC_df = vis_df.iloc[:, :-2].apply(lambda row: makeCArray(row.values), axis=1)
    visC_df = pd.DataFrame(visC_df.values.tolist(), index=visC_df.index)
    visC_tint_df = visC_df.loc[pd.IndexSlice[:, test_tint], :].droplevel(level=1)
    visamp_tint_df = np.abs(visC_tint_df)
    visphase_tint_df = visC_tint_df.apply(np.angle)
if coords == 'polar':
    visamp_tint_df = vis_df.iloc[:, :-2:2].loc[pd.IndexSlice[:, test_tint], :].\
                                               droplevel(level=1)
    visphase_tint_df = vis_df.iloc[:, 1:-2:2].loc[pd.IndexSlice[:, test_tint], :].\
                                                  droplevel(level=1)

In [ ]:
piv = pd.pivot_table(visamp_tint_df, columns='freq')
vmax = np.nanpercentile(piv.values, 95)
vmin = np.nanpercentile(piv.values, 5)
df_heatmap(piv, xbase=50, ybase=5, vmax=vmax, vmin=vmin, \
           title='Visibility amplitudes for time_integration {}'.format(test_tint), \
           xlabel='Frequency', \
           ylabel='Redundant Baseline Group')

In [ ]:
piv = pd.pivot_table(visphase_tint_df, columns='freq')
df_heatmap(piv, xbase=50, ybase=5, center=0, cmap='bwr', vmin=-np.pi, vmax=np.pi,\
           title='Visibility phases for time_integration {}'.format(test_tint), \
           xlabel='Frequency', \
           ylabel='Redundant Baseline Group')

##### Tilt shifts

Calculate tilt shifts of relatively calibrated results

In [ ]:
antpos_arr = flt_ant_pos(md['antpos'], np.unique(md['redg'][:, 1:]))

In [ ]:
gphases_df['x_tilt'] = gphases_df.apply(lambda row: np.sum(row.values[:no_ants]*antpos_arr[:, 0]), axis=1)
gphases_df['y_tilt'] = gphases_df.apply(lambda row: np.sum(row.values[:no_ants]*antpos_arr[:, 1]), axis=1)

In [ ]:
plot_res(gphases_df, 'x_tilt')

In [ ]:
plot_res(gphases_df, 'y_tilt')

In [ ]:
plot_res_heatmap(gphases_df, 'x_tilt')

In [ ]:
plot_res_heatmap(gphases_df, 'y_tilt')